In [ ]:
import copy # copy variables
import os   # manage paths

import uproot   # use of root files
import awkward as ak    # nested, variable sized data
import vector   # lorentz vectors
vector.register_awkward()
import matplotlib.pyplot as plt # plotting
import atlasopenmagic as atom

from src.parse_atlas import (parser, consts, schemas, combinatorics)

import traceback
%load_ext autoreload
%autoreload 2

In [ ]:
#TESTING MC files
mc_rec_id = atlasparser.fetch_mc_files_ids('2024', is_random=True)
all_metadata = atom.get_metadata(mc_rec_id)
urls = atom.get_urls(mc_rec_id)
atlasparser.parsing_pipeline(schemas.GENERIC_SCHEMA, files_ids=urls)

In [ ]:
#TESTING MAIN PIPELINE
import main_pipeline

main_pipeline.run()

In [ ]:
#TEST FOR KEYS OF ROOT FILE
from src.parse_atlas import parser, schemas
import atlasopenmagic as atom
parse = parser.ATLAS_Parser()
rand_id = parse.fetch_mc_files_ids(
    '2024', is_random=True)
uri = atom.get_urls(rand_id)[0]
# testing = parse.testing_load_file_as_object(uri)
testing = parse._parse_file(
    schema=schemas.GENERIC_SCHEMA, 
    file_index=uri)

In [ ]:
dir(testing)
testing.fields

In [ ]:
GeV = 1000.

def selected_electrons(el):
    return el[(el.pt > 10 * GeV) & (abs(el.eta) < 2.47)]

def selected_muons(mu):
    return mu[(mu.pt > 10 * GeV) & (abs(mu.eta) < 2.47)]

def selected_jets(j):
    return j[(j.pt > 10 * GeV) & (abs(j.eta) < 2.47)]

def no_overlap(obj1, obj2, deltaR=0.4):
    obj1, obj2 = ak.unzip(ak.cartesian([obj1, obj2], nested=True))
    return ak.all(obj1.deltaR(obj2) > deltaR, axis=-1)

def mjjj(jets):
    candidates = ak.combinations(jets, 3)
    j1, j2, j3 = ak.unzip(candidates)
    candidates["p4"] = j1 + j2 + j3
    candidates = candidates[ak.argmax(candidates.p4.pt, axis=1, keepdims=True)]
    return candidates.p4.mass

def processed(events):
    events = copy.copy(events) # shallow copy
    events["Electrons"] = selected_electrons(events.Electrons)
    events["Muons"] = selected_muons(events.Muons)
    events["Jets"] = selected_jets(events.Jets)
    events["Jets"] = events.Jets[no_overlap(events.Jets, events.Electrons)]
    events = events[
        (ak.num(events.Jets) >= 4) # at least 4 jets
        & ((ak.num(events.Electrons) + ak.num(events.Muons)) == 1) # exactly one lepton
    ]
    return ak.to_packed(events)

events = processed(atlasparser.events)
plt.hist(ak.flatten(mjjj(events.Jets) / GeV, axis=None), bins=100)
plt.xlabel("Reconstructed Top Quark Mass (GeV)")
plt.ylabel("Number of Events")
plt.title("Distribution of Reconstructed Top Quark Mass")
plt.axvline(172.76, color='r', linestyle='dashed', linewidth=2, label='Expected Top Quark Mass')
plt.legend()
plt.show()

print('Total events:', len(atlasparser.events))
print('Events after filtering:', len(events))

In [ ]:
def selected_electrons(el):
    return el[(el.pt > 10 * GeV) & (abs(el.eta) < 2.47)]

def electron_posi_muon_antimuon(events):
    events = copy.copy(events) # shallow copy
    events["Electrons"] = selected_electrons(events.Electrons)
    events["Muons"] = selected_muons(events.Muons)
    events["Electrons", "is_neg"] = events.Electrons.charge < 0
    events["Muons", "is_neg"] = events.Muons.charge < 0
    events = events[
        (ak.num(events.Electrons) == 2) 
        & (ak.num(events.Muons) == 2)
        # & (ak.num(events.Electrons[events.Electrons.is_neg]) == 1)
        # & (ak.num(events.Muons[events.Muons.is_neg]) == 1)
    ]
    return ak.to_packed(events)

events = electron_posi_muon_antimuon(atlasparser.events)
plt.hist(ak.flatten(events) / GeV, axis=None, bins=100)
plt.xlabel("Reconstructed Top Quark Mass (GeV)")
plt.ylabel("Number of Events")
plt.title("Distribution of Reconstructed Top Quark Mass")
plt.axvline(172.76, color='r', linestyle='dashed', linewidth=2, label='Expected Top Quark Mass')
plt.legend()
plt.show()

print('Total events:', len(atlasparser.events))
print('Events after filtering:', len(events))

In [2]:
import logging
import sys
from src.parse_atlas import parser, combinatorics, consts, schemas
import matplotlib.pyplot as plt # plotting
import awkward as ak
import tqdm

release_years = ['2016', '2020', '2024', '2025']

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)

def run():
    atlasparser = parser.ATLAS_Parser()
    release_files_uris = atlasparser.fetch_real_records_ids(release_year='2024')
    # atlasparser.parsing_pipeline(file_uris=release_files_uris)

    categories = combinatorics.make_objects_categories(consts.PARTICLE_LIST, min_n=2, max_n=4)

    for events_chunk in atlasparser.parse_files(files_ids=release_files_uris, limit=30):
        for category in categories:
            logging.info(f"Processing category: {category}")
            combination_dict = combinatorics.make_objects_combinations_for_category(
                    category, min_k=2, max_k=4)
            
            #IF CAN FILTER ACCORDING TO ITERATION'S COMBINATION
            if not all(obj in events_chunk.fields for obj in combination_dict.keys()):
                logging.info('Not all of the combination objects are present in the events chunk. ')
                continue    

            # combo_events = atlasparser.filter_events_by_combination(
            #     events_chunk, combination_dict)

            combination_events_mass = atlasparser.calculate_mass_for_combination(events_chunk)
            
            #COMBO_EVENTS IS THE EVENTS FILTERED FOR EACH COMBINTATION
            #NEXT STEP, MAKE A MASS HIST OUT OF IT
            plt.hist(ak.flatten(combination_events_mass / consts.GeV, axis=None), bins=100)
            plt.xlabel("Reconstructed Top Quark Mass (GeV)")
            plt.ylabel("Number of Events")
            plt.title("Distribution of Reconstructed Top Quark Mass")
            plt.axvline(172.76, color='r', linestyle='dashed', linewidth=2, label='Expected Top Quark Mass')
            plt.legend()
            plt.show()
            0/0
            

            # for combination in combinations:
            #     combination_dict = {
            #         obj[:-1]: int(obj[-1]) for obj in combination}

            #     #IF CAN FILTER ACCORDING TO ITERATION'S COMBINATION
            #     if not all(obj in events_chunk.fields for obj in combination_dict.keys()):
            #         logging.info('Not all of the combination objects are present in the events chunk. ')
            #         continue    

            #     combo_events = atlasparser.filter_events_by_combination(
            #         events_chunk, combination_dict)
                
            #     #COMBO_EVENTS IS THE EVENTS FILTERED FOR EACH COMBINTATION
            #     #NEXT STEP, MAKE A MASS HIST OUT OF IT
            #     plt.hist(ak.flatten(combo_events / consts.GeV, axis=None), bins=100)
            #     plt.xlabel("Reconstructed Top Quark Mass (GeV)")
            #     plt.ylabel("Number of Events")
            #     plt.title("Distribution of Reconstructed Top Quark Mass")
            #     plt.axvline(172.76, color='r', linestyle='dashed', linewidth=2, label='Expected Top Quark Mass')
            #     plt.legend()
            #     plt.show()
run()

INFO:root:Total amount of files found: 55620


Starting to parse 30 files with 3 threads.


Parsing files:   0%|          | 0/30 [00:00<?, ?file/s]

Parsing files:   3%|▎         | 1/30 [00:13<06:27, 13.36s/file, 48.00 KB | root://eospublic.cern.ch//eos/opendata/atlas/rucio/data15_13TeV/DAOD_PHYSLITE.37001626._000003.pool.root.1]


Parsed file with objects: ['Electrons', 'Muons', 'Jets', 'Photons', 'MET']
Yielding chunk with 10374 events after 1 files.


Exception ignored in: <generator object ATLAS_Parser.parse_files at 0x7f668bf948e0>
Traceback (most recent call last):
  File "/workspaces/opendata_ddp/src/parse_atlas/parser.py", line 89, in parse_files
    
  File "/usr/lib/python3.13/concurrent/futures/_base.py", line 647, in __exit__
    self.shutdown(wait=True)
  File "/usr/lib/python3.13/concurrent/futures/thread.py", line 239, in shutdown
    t.join()
  File "/usr/lib/python3.13/threading.py", line 1092, in join
    self._handle.join(timeout)
KeyboardInterrupt: 


AttributeError: 'generator' object has no attribute 'keys'